# Projet maison n° 4

In [1]:
# imports
import re
import requests
from bs4 import BeautifulSoup

**Partie A**

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

```python
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"
```

Avec `URL_PAGE2`, la fonction doit retourner :

```json
{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
```

In [2]:
# partie A
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"

def get_prices_from_url(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content)
    
    prices = {}
    
    # add headings to dictionary
    for heading2 in soup.find_all('h2'):
        prices[heading2.text] = {}
    
    # add prices to dictionary
    heading_index = -1
    for item in soup.find_all('span', attrs={'class':"pricing-table-price"}):
        heading_index = heading_index + 1
        prices[list(prices.keys())[heading_index]]['price'] = '$' + re.search('([0-9]+)',item.text).group(0)
        
    # add storage and databases to dictionary
    heading_index = -1
    for ul_item in soup.find_all('ul', attrs={'class':"pricing-table-list"}):
        heading_index = heading_index + 1

        for li_item in ul_item.find_all('li'):

            storage = re.search('[0-9]+(G|T)B file storage', li_item.text)
            databases = re.search('[0-9]+ database(s)?', li_item.text)
            if storage is not None:
                storage_unit = re.search('[0-9]+(G|T)B', storage.group(0))
                storage_unit = re.search('[A-Za-z]+', storage_unit.group(0)).group(0)
                prices[list(prices.keys())[heading_index]]['storage'] = re.search('[0-9]+', storage.group(0)).group(0) + storage_unit
        
            if databases is not None:
                prices[list(prices.keys())[heading_index]]['databases'] = int(re.search('[0-9]+', databases.group(0)).group(0))
    
    # sort each sub_dictionary
    for i in range(len(prices)):
        prices[list(prices.keys())[i]] = dict(sorted(prices[list(prices.keys())[i]].items()))
        
    return dict(prices)

In [3]:
# test get_prices_from_url(url)
get_prices_from_url(URL_PAGE3)

{'Personal': {'databases': 1, 'price': '$5', 'storage': '1GB'},
 'Small Business': {'databases': 5, 'price': '$25', 'storage': '10GB'},
 'Enterprise': {'databases': 25, 'price': '$45', 'storage': '100GB'},
 'Privilege': {'databases': 100, 'price': '$99', 'storage': '1TB'}}

**Partie B**

*L'abus d'alcool est dangereux pour la santé, à consommer avec modération.*

1) Ecrire une fonction `extract_beer_infos()` qui extrait des informations sur une bière du site de bières *beowulf*.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/cuvee-des-trolls.33 

La fonction doit retourner :
```json
{'Nom': 'Cuvée des Trolls',
 'Style': 'Bière Blonde',
 'Contenu': 25,
 'Degré d’alcool': 7.0,
 'Origine': 'Belgique',
 'Brasseur': 'Brasserie Dubuisson Freres'}
```

2) L'URL ci-après retourne un JSON avec une liste de bières :

```python
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"
```

Ecrire une fonction `extract_beer_list_infos(url)` qui prend en argument cet URL et retourne les informations sur une liste de bières du site *beowulf*.

Cette fonction doit retourner la liste des informations obtenues par la fonction précédemment définie `extract_beer_infos()`.

Exemple de retour :

```json
[{'Nom':'Desperados','Style':'Lager','Contenu':33,'Degré d’alcool':5.9,'Origine':'France','Brasseur':'Desperados'},
{'Nom':'La Lager Sans Gluten de Vézelay','Style':'Lager','Contenu':25,'Degré d’alcool':4.0,'Origine':'France','Brasseur':'Brasserie de Vézelay'},
{'Nom':'Mélusine Bio','Style':'Pale Ale','Contenu': 33,'Degré d’alcool': 5.0,'Origine':'France','Brasseur':'Mélusine'},
{'Nom':'La Parisienne Le Titi Parisien','Style':'IPA','Contenu':33,'Degré d’alcool':5.5,'Origine':'France','Brasseur': 'Brasserie la Parisienne'},
{'Nom':'Brasserie De Sutter Brin de Folie','Style':'Bière Blonde','Contenu': 33,'Degré d’alcool':6.5,'Origine':'France','Brasseur':'Brasserie de Sutter'}]
```

**Facultatif**

Chercher comment optimiser cette fonction en utilisant `multiprocessing.Pool()` pour paralléliser les accès web.

In [4]:
url_partie_b1 = 'https://www.beerwulf.com/fr-fr/p/bieres/gallia-west-ipa-fut--2l'
# partie B-1
def extract_beer_infos(url):
    beer_infos = {
        'Nom': None,
        'Style': None,
        'Contenu': None,
        'Degré d’alcool': None,
        'Origine': None,
        'Brasseur': None
    }
    
    # get url information
    response = requests.get(url)
    soup = BeautifulSoup(response.content)

    
    # Nom
    for item in soup.find_all('h1', attrs={'class':"product-name"}):
        beer_infos['Nom'] = item.text
        
    for item in soup.find_all('td', attrs={'class':"header"}):
        
        if item.text == 'Style':
            beer_infos['Style'] = item.find_next().text
        
        elif item.text == 'Contenu':
            beer_infos['Contenu'] = int(re.search('[0-9]+', item.find_next().text).group(0))
        
        elif item.text == "Degré d’alcool":
            beer_infos['Degré d’alcool'] = float(re.search('[0-9]+(,[0-9]+)?', item.find_next().text).group(0).replace(',','.'))
        
        elif item.text == 'Origine':
            beer_infos['Origine'] = re.search('[A-Za-z]+',item.find_next().text).group(0)
        
        elif item.text == 'Brasseur':
            beer_infos['Brasseur'] = re.search('[A-Za-z]+', item.find_next().text).group(0)

    return beer_infos

In [5]:
# Test extract_beer_infos(url)
extract_beer_infos(url_partie_b1)

{'Nom': 'Gallia West IPA - Fût 2L The SUB',
 'Style': 'IPA',
 'Contenu': 2,
 'Degré d’alcool': 6.0,
 'Origine': 'France',
 'Brasseur': 'Gallia'}

In [6]:
# partie B-2
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"
URL_HOME = 'https://www.beerwulf.com'

def extract_beer_list_infos(url):
    # Collecter les pages de bières à partir du JSON
    r = requests.get(URL_BEERLIST_FRANCE)
    body = r.json()

    beer_pages = []
    
    for i in range(len(body['items'])):
        beer_pages.append(body['items'][i]['contentReference'])
        
    # Sequential version (slow):
    beers = []
    for page in beer_pages:
        beers.append(extract_beer_infos(URL_HOME + page))
        
    # Facultatif
    # Parallel version (faster):
    # beers = []
    
    return beers

In [7]:
# test extract_beer_list_infos
extract_beer_list_infos(URL_BEERLIST_FRANCE)

[{'Nom': 'La Cristal IPA du Mont Blanc',
  'Style': 'IPA',
  'Contenu': 33,
  'Degré d’alcool': 4.7,
  'Origine': 'France',
  'Brasseur': 'Mont'},
 {'Nom': 'Ninkasi Pale Ale',
  'Style': 'Bière Ambrée',
  'Contenu': 33,
  'Degré d’alcool': 4.5,
  'Origine': 'France',
  'Brasseur': 'Ninkasi'},
 {'Nom': 'Desperados',
  'Style': 'Lager',
  'Contenu': 33,
  'Degré d’alcool': 5.9,
  'Origine': 'France',
  'Brasseur': 'Desperados'},
 {'Nom': 'Pietra',
  'Style': 'Lager',
  'Contenu': 33,
  'Degré d’alcool': 6.0,
  'Origine': 'France',
  'Brasseur': 'Brasserie'}]

In [8]:
import unittest

class Session4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        self.assertIsInstance(infos, list)
        self.assertGreater(len(infos), 1)
        # Contenu = int
        # Degré d’alcool = float
        for beer in infos:
            self.assertIsInstance(beer['Nom'], str)
            self.assertIsInstance(beer['Style'], str)
            self.assertIsInstance(beer['Contenu'], int)
            self.assertIsInstance(beer['Degré d’alcool'], float)
            self.assertEqual(beer['Origine'], "France")
            self.assertIsInstance(beer['Brasseur'], str)

            
def run_tests():
    test_suite = unittest.makeSuite(Session4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [9]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Session4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Session4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Session4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 1.379s

OK
